[COVID-19 and RSV Monitoring Dashboard] (C) Astria Gonsalves, 2025 (a.a.gonsalves@se25.qmul.ac.uk).
Based on UK Government [data](https://ukhsa-dashboard.data.gov.uk/) published by the [UK Health Security Agency](https://www.gov.uk/government/organisations/uk-health-security-agency) and on the [DIY Disease Tracking Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) by Fabrizio Smeraldi. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).

# COVID-19 and RSV Monitoring Dashboard

## About Dashboard

The RSV and Covid-19 Monitoring dashboard is an interactive tracking dashboard that provides a simple and current overview of Respiratory Syncytial Virus (RSV) hospital admissions and RSV and COVID-19 PCR positivity testing trends over time in England. It is designed to support public health monitoring, analyse trends, and compare seasonal patterns and over multiple years. By combining hospital admission data with positivity testing data, the dashboard allows users to observe how infection trends relate to disease severity and hospital burden.

Data is presented through graphs and charts with all visualisations up-to-date ensuring that users have access to the most current available data.
The most recent data that is displayed on the dashboard is enabled through the interaction of the button named "Get current data" allowing the user to refresh the graphs and sync with the latest available information.

In [1]:
# Import
import requests
import time
import json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from IPython.display import clear_output
import ipywidgets as wdg



%matplotlib inline
plt.rcParams['figure.dpi'] = 100

In [2]:
# API wrapper 
class APIwrapper:
    _access_point="https://api.ukhsa-dashboard.data.gov.uk"
    _last_access=0.0 # time of last api access
    
    def __init__(self, theme, sub_theme, topic, geography_type, geography, metric):
        """ Init the APIwrapper object, constructing the endpoint from the structure
        parameters """
        
        url_path=(f"/themes/{theme}/sub_themes/{sub_theme}/topics/{topic}/geography_types/" +
                  f"{geography_type}/geographies/{geography}/metrics/{metric}")
        
        self._start_url=APIwrapper._access_point+url_path
        self._filters=None
        self._page_size=-1
        self.count=None

    def get_page(self, filters={}, page_size=5):        
        if page_size>365:
            raise ValueError("Max supported page size is 365")
        
        if filters!=self._filters or page_size!=self._page_size:
            self._filters=filters
            self._page_size=page_size
            self._next_url=self._start_url
        
        if self._next_url==None: 
            return []
        
        curr_time=time.time()
        deltat=curr_time-APIwrapper._last_access
        if deltat<0.33:
            time.sleep(0.33-deltat)
        APIwrapper._last_access=curr_time
        
        parameters={x: y for x, y in filters.items() if y!=None}
        parameters['page_size']=page_size
        retry=0
        response = requests.get(self._next_url, params=parameters).json()
        self._next_url=response['next']
        self.count=response['count']
        return response['results'] 
    def get_all_pages(self, filters={}, page_size=365):
        data=[] 
        while True:
            
            next_page=self.get_page(filters, page_size)
            if next_page==[]:
                break 
            data.extend(next_page)
        return data


In [3]:
# LOADING AND WRANGLING DATA:

In [4]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

In [5]:
#to visualize data:
#first graph = COVID-19 + RSV = positive (TIME SERIES GRAPH)
#second graph = RSV = rsv_admissions by age (BAR-HISTOGRAM CHART)

def parse_data():
    with open("rsv_positive.json", "rt") as INFILE:
        rsv_positive=json.load(INFILE)
    with open("covid_positive.json", "rt") as INFILE:
        covid_positive=json.load(INFILE)
    with open("rsv_admissions.json", "rt") as INFILE:
        rsv_admissions=json.load(INFILE)


    # POSITIVE DATA

    metadata={}
    for mydata in [rsv_positive, covid_positive]:
        for entry in mydata:
            date=entry['date']
            metric=entry['metric']
            value=entry['metric_value']
            if date not in metadata:
                metadata[date]={}
            metadata[date][metric]=value

    dates=list(metadata.keys())
    dates.sort()

    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    #print(startdate, 'to', enddate)


    index=pd.date_range(startdate, enddate, freq='D')
    positive_df=pd.DataFrame(index=index, columns=['rsv_positive', 'covid_positive'])
    positive_df


    metrics ={'rsv_positive': 'RSV_testing_positivityByWeek',
            'covid_positive': 'COVID-19_testing_positivity7DayRolling'}

    for date, entry in metadata.items():
        pd_date=parse_date(date)
        for column in metrics.keys(): 
            metric_name=metrics[column]
            value= entry.get(metric_name, 0.0)
            positive_df.loc[date, column]=value
            
    positive_df.fillna(0.0, inplace=True)

    #ax=positive_df.plot()
    #ax.set_title('Positive RSV tests, Positive COVID-19 tests')

    #-old-ax=positive_df.plot(logy=True)
    #-old-ax.set_title('Positive RSV tests, Positive COVID-19 tests (log scale)');


    # RSV_ADMISSIONS DATA

    metadata={}
    for entry in rsv_admissions:
        date=entry['date']
        age=entry['age']
        value=entry['metric_value']
        if date not in metadata:
            metadata[date]={}
        metadata[date][age]=value


    dates=list(metadata.keys())
    dates.sort()
    dates


    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    #print(startdate, 'to', enddate)

    age=[]
    for entry in metadata.values():
        for xages in entry.keys():
            if xages not in age:
                age.append(xages)
    age.sort()
    #print(age)

    index=pd.date_range(startdate, enddate, freq='M')
    rsv_admissions_df=pd.DataFrame(index=index, columns=age)
    rsv_admissions_df


    for date, entry in metadata.items():
        pd_date=parse_date(date)
        for column in entry.keys():
            rsv_admissions_df.loc[date, column]=entry[column]


    rsv_admissions_df.fillna(0.0, inplace=True)

    rsv_admissions_df.infer_objects(copy=False)   

    quarterly= rsv_admissions_df.groupby(pd.Grouper(freq='1QE')).mean() 
    totals=quarterly.sum(axis=1)
    quarterly=quarterly.div(totals, axis=0)*100
    quarterly = quarterly[::-1]


    #ax=positive_df.plot()
    #ax.set_title('Positive RSV tests, Positive COVID-19 tests')
    #-old-ax=positive_df.plot(logy=True)
    #-old-ax.set_title('Positive RSV tests, Positive COVID-19 tests (log scale)');

    #ax=quarterly.plot(kind='barh', stacked=True,cmap='tab20')
    #ax.legend(loc='center left',bbox_to_anchor=(1.0, 0.5))

    #ax.set_yticklabels(quarterly.index.strftime('%Y-%m-%d'))
    #ax.set_title('Lineage prevalence by quarter ending');


    positive_df.to_pickle("positive_df.pkl")
    rsv_admissions_df.to_pickle("rsv_admissions_df.pkl")
    positive_df=pd.read_pickle("positive_df.pkl")
    rsv_admissions_df=pd.read_pickle("rsv_admissions_df.pkl")
    return positive_df, rsv_admissions_df

In [6]:
# DOWNLOADING DATA

In [7]:
def get_data():

    #COVID_POSITIVE structure
    covid_positive_structure={"theme": "infectious_disease", 
            "sub_theme": "respiratory",
            "topic": "COVID-19",
            "geography_type": "Nation", 
            "geography": "England"}
    covid_positive_structure['metric']='COVID-19_testing_positivity7DayRolling'

    # Filter
    filters={"stratum" : None,
            "age": None,
            "sex": None,
            "year": None,
            "month": None,
            "epiweek" :None,
            "date" : None,
            "in_reporting_delay_period": None
            }

    #covid_positive API object creation
    covid_positive_api=APIwrapper(**covid_positive_structure)
    # covid_positive data
    covid_positive=covid_positive_api.get_all_pages()
    #print(covid_positive_api.count)
    #print(covid_positive)
    
    with open("covid_positive.json", "wt") as file:
        json.dump(covid_positive, file, indent=4)


    
    #RSV_POSITIVE structure
    rsv_positive_structure={"theme": "infectious_disease", 
            "sub_theme": "respiratory",
            "topic": "RSV",
            "geography_type": "Nation", 
            "geography": "England"}
    rsv_positive_structure['metric']='RSV_testing_positivityByWeek'
    rsv_positive_api=APIwrapper(**rsv_positive_structure)
    rsv_positive = rsv_positive_api.get_all_pages()
    #print(rsv_positive[:5])
    #print(f"Data points expected: {rsv_positive_api.count}")
    #print(f"Data points retrieved: {len(rsv_positive)}")

    with open("rsv_positive.json", "wt") as OUT:
        json.dump(rsv_positive, OUT, indent=4)

    

    #RSV_ADMISSIONS structure
    rsv_admissions_structure={"theme": "infectious_disease", 
            "sub_theme": "respiratory",
            "topic": "RSV",
            "geography_type": "Nation", 
            "geography": "England"}
    rsv_admissions_structure['metric']='RSV_healthcare_admissionRateByWeek'
    rsv_admissions_api=APIwrapper(**rsv_admissions_structure)
    rsv_admissions = rsv_admissions_api.get_all_pages()
    #print(rsv_admissions[:5])
    #print(f"Data points expected: {rsv_admissions_api.count}")
    #print(f"Data points retrieved: {len(rsv_admissions)}")


    with open("rsv_admissions.json", "wt") as OUT:
        json.dump(rsv_admissions, OUT, indent=4)

In [8]:
# Access API Button
def access_api(button):
    apibutton.icon="spinner"
    apibutton.description = "Loading..."
    try:
        get_data()
        global positive_df, rsv_admissions_df
        positive_df, rsv_admissions_df = parse_data()
        refresh_graphs()
        apibutton.icon="check"
        apibutton.description = "Data Refreshed"
        apibutton.disabled=False
    except Exception as e:
        #print("Cannot get refresh data, using existing data")
        apibutton.description='Cannot Refresh Data'

In [9]:
# Create Refresh Button
apibutton=wdg.Button(
    description='Get current data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download latest data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)

# Error Handling when no json files are found
try:
    positive_df, rsv_admissions_df = parse_data()
except:
    get_data()
    positive_df, rsv_admissions_df = parse_data()

# callback function button
apibutton.on_click(access_api)
display(apibutton)


/tmp/ipykernel_3960/3410406229.py:49: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  positive_df.fillna(0.0, inplace=True)
/tmp/ipykernel_3960/3410406229.py:87: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  index=pd.date_range(startdate, enddate, freq='M')
/tmp/ipykernel_3960/3410406229.py:98: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  rsv_admissions_df.fillna(0.0, inplace=True)


Button(description='Get current data', icon='download', style=ButtonStyle(), tooltip='Click to download latest…

In [10]:
# GRAPHS AND ANALYSIS

In [11]:
# Time Series Graph for RSV and COVID-19 testing - weekly
def positive_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        pos_ax=positive_df[list(gcols)].plot(logy=logscale)
        pos_ax.legend(loc='center left',bbox_to_anchor=(1.0, 0.5))
        pos_ax.set_xlabel('Time (Years)')
        pos_ax.set_ylabel('Quantity of Positive Tests')
        plt.show()
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

In [12]:
# Bar-Histogram Graph for RSV Admissions - Monthly 
def rsv_admissions_graph(graphyear):
    yeardf=rsv_admissions_df[rsv_admissions_df.index.year==graphyear]

    monthly= yeardf.groupby(pd.Grouper(freq='1ME')).mean() 
    totals=monthly.sum(axis=1)
    monthly=monthly.div(totals, axis=0)*100
    monthly = monthly[::-1]

    admit_ax=monthly.plot(kind='barh', stacked=True,cmap='tab20') 
    admit_ax.legend(loc='center left',bbox_to_anchor=(1.0, 0.5))
    admit_ax.set_yticklabels(monthly.index.strftime('%Y-%m-%d'))
    admit_ax.set_xlabel('Normalised Admissions Rate')
    admit_ax.set_ylabel('Time (Months)')

    plt.show()

In [13]:
# Refresh Graphs
def refresh_graphs():
    # Triggers graph re-draw for RSV+Covid positivity data refresh
    scale_current = scale.value
    if scale_current == 'linear':
        other = 'log'
    else:
        other = 'linear'
    scale.value = other
    scale.value = scale_current  # Revert back

    # Trigger re-draw for graph RSV admissions data refresh
    year_value = year.value
    if year_value == year.options[0]:
        year_other = year.options[1]
    else:
        year_other = year.options[0]
    year.value = year_other
    year.value = year_value  # Revert back

## PCR Testing

### Positive PCR Tests for RSV and for COVID-19

The graph displays the total number of positive PCR tests for RSV and for COVID-19 over the years as well as a comparison between the two respiratory diseases over time when both diseases are selected. The graphs can be viewed on a linear or log scale, defaulted to linear.

In [14]:
# RSV and COVID Positive Widget
tests=wdg.SelectMultiple(
    options=['rsv_positive', 'covid_positive'],
    value=['rsv_positive', 'covid_positive'],
    rows=2,
    description='Diseases:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)

controls=wdg.HBox([tests, scale])

 
graph=wdg.interactive_output(positive_graph, {'gcols': tests, 'gscale': scale})

display(controls, graph)
#time.sleep(1)
#refresh_graphs()

Output()

## Admissions

### Monthly RSV Admissions by Age Group

The chart displays the total number of people admitted to hospital with confirmed RSV every month over the years. The chart allows comparison between age groups over time as well as comparisons of seasonal patterns over the years through the drop down list of specific years.

In [15]:
# RSV Admissions Widget
year=wdg.Select(
    
    options=rsv_admissions_df.index.year.unique(),
    value=rsv_admissions_df.index.year[-1],
    rows=1,
    description='Year',
    disabled=False
)
 
output=wdg.interactive_output(rsv_admissions_graph, {'graphyear': year})

display(year, output)

Select(description='Year', index=5, options=(2020, 2021, 2022, 2023, 2024, 2025), rows=1, value=2025)

Output()

**Author and License** [COVID-19 and RSV Monitoring Dashboard] (C) Astria Gonsalves, 2025 (a.a.gonsalves@se25.qmul.ac.uk).
Based on UK Government [data](https://ukhsa-dashboard.data.gov.uk/) published by the [UK Health Security Agency](https://www.gov.uk/government/organisations/uk-health-security-agency) and on the [DIY Disease Tracking Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) by Fabrizio Smeraldi. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).